# Derivas de concepto en la mempool. Método ADWIN

Detección de derivas de concepto en la media de transacciones de la mempool por intervalo de tiempo, a partir del método matemático ADWIN, con salida gráfica.

In [1]:
!pip install river

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from btc_connect import btc_connect
from bitcoinrpc.authproxy import JSONRPCException
from river import drift
from datetime import datetime
import time, numpy as np, matplotlib.pyplot as plt

In [3]:
# Conección al nodo local
rpc_connection = btc_connect()

In [4]:
# Adaptación de función para graficar el total de transacciones de la mempool y las derivas de concepto detectadas
# https://riverml.xyz/latest/introduction/getting-started/concept-drift-detection/
def plot_transaction(x, y, xlabel, drift=None):
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(x, y, label="transacciones")
    ax.set_ylim(bottom=0)
    ax.set_title("Número de transacciones pendientes")
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Transacciones")
    if drift is not None:
        legend=0
        for drift_detected in drift:
            if legend==0:
                ax.axvline(drift_detected, color="red", linestyle=":", label="derivas de concepto")
                legend=1
            else:
                ax.axvline(drift_detected, color="red", linestyle=":")
    fig.legend(loc="outside upper left")
    plt.show()

In [5]:
mempool_temp_transactions = []
mempool_iteration = []
mempool_datetime = []
mempool_transactions = []

# ADWIN Valores por defecto:
# https://riverml.xyz/dev/api/drift/ADWIN/
# delta=0.002         -> Valor de significancia 
# clock=32            -> Frecuencia con la que se comprueban cambios. 1 => con cada nuevo dato
# max_buckets=5       -> Cantidad máxima de contenedores de datos de cada tamaño que se debe mantener, como paso previo a ser fusionados
# min_window_length=5 -> Longitud mínima que debe tener una subventana para ser evaluada 
# grace_period=10     -> Cantidad mínima de datos exigida para realizar la detección de cambios

drift_detector = drift.ADWIN(delta=0.000002)

mempool_iteration_drift = [] # Iteración en la que se detecta una deriva de concepto en la mempool
mempool_datetime_drift = []  # Instante temporal en el que se detecta una deriva de concepto en la mempool
mempool_transactions_drift = []  # Número total de transacciones que activa una deriva

start=""
i = 0

observations = 60
repetitions = 720  # 12 horas
seconds = 1

while i < repetitions:
    j = 0
    mempool_temp_transactions.clear()
    while j < observations:
        try:
            # Obtiente la mempool del nodo local
            mempool = rpc_connection.getmempoolinfo()
        except JSONRPCException as e:
            print(f"Error al obtener la mempool del nodo local: {e}")
            mempool = {}
    
        timestamp = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
        if start == "":
            start = timestamp
    
        mempool_temp_transactions.append(mempool["size"])            
        j+=1
        # Tiempo para realizar la siguiente solicitud
        time.sleep(seconds)
    
    mempool_iteration.append(i)
    mempool_datetime.append(timestamp)
    mempool_mean = np.mean(mempool_temp_transactions)
    mempool_transactions.append(mempool_mean)
    print(mempool_mean)
    
    drift_detector.update(mempool_mean)
    if drift_detector.drift_detected:  # Deriva de concepto detectada
        mempool_iteration_drift.append(i)
        mempool_datetime_drift.append(timestamp)
        mempool_transactions_drift.append(mempool_mean)
    i+=1
   
end = timestamp
xlabel = f"Inicio: {start} - Fin: {end}. Número de iteraciones: {i}"

87353.81666666667
87573.0
87833.96666666666
88013.31666666667
87599.01666666666
86420.53333333334
86714.93333333333
86988.63333333333
87279.18333333333
87537.71666666666
87764.73333333334
87970.03333333334
88218.08333333333
88476.66666666667
87360.53333333334
86791.66666666667
87085.1
87372.81666666667
87607.91666666667
87815.21666666666
88015.66666666667
88234.33333333333
88457.55
88728.65
88933.83333333333
87438.13333333333
87235.93333333333
87506.93333333333
87861.58333333333
88104.56666666667
88378.38333333333
88590.75
88802.46666666666
89005.83333333333
89185.73333333334
89403.86666666667
89653.21666666666
89930.4
90112.91666666667
90323.56666666667
90585.06666666667
90809.86666666667
91041.31666666667
91244.41666666667
91496.91666666667
91771.96666666666
92031.6
92323.78333333334
92691.85
93031.83333333333
93259.31666666667
93574.61666666667
93907.58333333333
94339.7
94612.88333333333
94867.61666666667
95124.48333333334
95463.56666666667
95758.3
96076.43333333333
94255.5166666666

TimeoutError: timed out

### Derivas de concepto detectadas

In [ ]:
plot_transaction(mempool_iteration, mempool_transactions, xlabel, mempool_iteration_drift)

In [ ]:
print(mempool_iteration)
print(mempool_transactions)

### Instante temporal en el que se detecta una deriva de concepto y número de transacciones

In [ ]:
for x, y in (zip(mempool_datetime_drift, mempool_transactions_drift)):
    print(f"Instante temporal {x}: {y} transacciones")